In [1]:
%reload_ext autoreload
%autoreload 2

In [25]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import load_pkl, GRAY_DASH, save_latex, FLAVOR_NAMES, ARCH_NAMES, texttt, footnotesize

data: DiffResult = load_pkl("config")

ISSUES = {
    IssueEnum.ADD: "$+$",
    IssueEnum.REMOVE: "$-$",
    IssueEnum.CHANGE: r"$\Delta$",
    IssueEnum.NEW: "#",
}

KINDS = {
    DepKind.STRUCT: "Struct",
    DepKind.CONFIG: "Config",
    DepKind.FUNC: "Func",
    DepKind.TRACEPOINT: r"\shortstack{Tracept\\($\Delta=0$)}",
    DepKind.SYSCALL: r"Syscall",
}

GROUPS = {
    VersionGroup.ARCH: "Architecture",
    VersionGroup.FLAVOR: "Flavor",
}


def format_val(v) -> str:
    if v > 1000:
        return f"{v/1000:.1f}k"
    return v


results = {}

def_pair_result = list(data.data[VersionGroup.ARCH].data.values())[0]
result = {}
for kind, kind_result in def_pair_result.iter_kinds():
    if kind not in KINDS:
        continue
    for issue, count in kind_result.iter_issues():
        if issue == IssueEnum.OLD:
            result[(KINDS[kind], ISSUES[IssueEnum.NEW])] = format_val(count)
        elif (
            issue in (IssueEnum.ADD, IssueEnum.REMOVE, IssueEnum.CHANGE)
            and kind != DepKind.CONFIG
        ):
            result[(KINDS[kind], ISSUES[issue])] = 0
result[("Register", ISSUES[IssueEnum.CHANGE])] = 0
results[(r"\multicolumn{1}{c|}{def-}", r"\multicolumn{1}{c|}{ault}")] = result


for group, group_result in data.iter_groups():
    if group not in GROUPS:
        continue
    for pair, pair_result in group_result.iter_pairs():
        result = {}
        for kind, kind_result in pair_result.iter_kinds():
            if kind not in KINDS:
                continue
            for issue, count in kind_result.iter_issues():
                if issue not in ISSUES:
                    continue
                if kind == DepKind.CONFIG and issue != IssueEnum.NEW:
                    continue
                result[(KINDS[kind], ISSUES[issue])] = format_val(count)
        result[("Register", ISSUES[IssueEnum.CHANGE])] = 1 if group == VersionGroup.ARCH else 0
        if group == VersionGroup.ARCH:
            name = texttt(footnotesize(ARCH_NAMES[pair.v2.arch]))
        elif group == VersionGroup.FLAVOR:
            name = FLAVOR_NAMES[pair.v2.flavor]
        results[(GROUPS[group], name)] = result


df = pd.DataFrame(results)
df = df.loc[(df != 0).any(axis=1)]


df = df.map(lambda x: GRAY_DASH if x == 0 else x)
latex = df.to_latex(
    multicolumn_format="c|", column_format=r"p{7ex}p{1.5ex}|r|rrrr|rrrr", multirow=True
)

save_latex(latex, "cfg", rotate=False)

[ utils_pickle.py:18 ] INFO: Loding config from /Users/szhong/Code/DepSurf/output/config.pkl
[  utils_latex.py:101] INFO: Saved cfg to /Users/szhong/Code/DepSurf/paper/tabs/cfg.tex
